In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mesa_reader as mr
from scipy.spatial import KDTree

############ PLEASE SET PATH TO HISTORY FILES #####
mesa_dirs = {"track_1":"/home/path/to/the/file"}

# Load stellar evolution tracks
stellar_tracks = {}
track_points = []  # Store (Teff, log_L, mass) for k-NN search
mass_labels = []   # Corresponding mass labels

for key, path in mesa_dirs.items():
    history = mr.MesaData(path)
    mass = history.data('star_mass')[0]  # Read mass from history file
    
    Teff_log = np.log10(history.data('Teff'))
    log_L = history.data('log_L')
    stellar_tracks[mass] = {"Teff": Teff_log, "log_L": log_L}
    
    # Store all points with mass labels for nearest-neighbor search
    track_points.extend(zip(Teff_log, log_L))
    mass_labels.extend([mass] * len(Teff_log))

# Convert to NumPy arrays
track_points = np.array(track_points)
mass_labels = np.array(mass_labels)

############## USER INPUT PULSATION PERIOD OF CEPHEID AND UNCERTAINTY #################
P = 16.01
del_P = 0.0001

############## USER INPUT CONSTANTS OF PL RELATION ALPHA, BETA, AND FE/H ##############
a = -2.715
b = -4.153
del_beta = 0.060  # Uncertainty in beta
FE_H = 0.01
################## INPUT BOLOMETRIC CORRECTION ##################
BC = -0.161  # User-specified bolometric correction

### PL RELATION CALCULATIONS ###
M_v = a * (np.log10(P) - np.log10(10)) + b + FE_H # Absolute magnitude in V-band
M_bol = M_v + BC  # Bolometric magnitude
L_bol = (4.8 - M_bol) / 2.5  # Luminosity log(L/L_sun)

# Uncertainty calculations
del_Mv = abs(a) * (del_P / P) + del_beta
del_L = abs((-M_bol * del_Mv) / 2.5)  # Error propagation

########### ENTER Teff AND UNCERTAINTY #########
Teff = 3.75
del_T = 0.01

########### OR ENTER B-V & UNCERTAINTY TO AUTOMATICALLY CALCULATE TEMP #########
B_V = 1.04
del_B_V = 0.02

fie = 0.716 + (0.225 * B_V)
del_fie = 0.015 + (B_V * 0.017) + del_B_V * 0.225

T_eff = np.log10(5040 / fie)
del_T_eff = abs((del_fie / fie**2) / T_eff)  # Error propagation for log Teff

# Use calculated Teff if B-V is given
if B_V is not None:
    Teff = T_eff
    del_T = del_T_eff

############## CEPHEID DATA ##############
cepheids = {
    "Cepheid": {"Teff": Teff, "Lum": L_bol, "del_T": del_T, "del_L": del_L, "Color": "pink"},
}

# Build k-NN tree for mass identification
tree = KDTree(track_points)

for name, data in cepheids.items():
    Teff_min, Teff_max = data["Teff"] - data["del_T"], data["Teff"] + data["del_T"]
    Lum_min, Lum_max = data["Lum"] - data["del_L"], data["Lum"] + data["del_L"]

    # Find all points within the box
    mask = (track_points[:, 0] >= Teff_min) & (track_points[:, 0] <= Teff_max) & \
           (track_points[:, 1] >= Lum_min) & (track_points[:, 1] <= Lum_max)

    masses_in_box = mass_labels[mask]

if len(masses_in_box) > 0:
    min_mass, max_mass = np.min(masses_in_box), np.max(masses_in_box)
    mass_central = (min_mass + max_mass) / 2  # Central estimate
    mass_upper = max_mass - mass_central  # Upper deviation
    mass_lower = mass_central - min_mass  # Lower deviation

    mass_range = f"${mass_central:.2f}^{{+{mass_upper:.2f}}}_{{-{mass_lower:.2f}}}$ M"
else:
    mass_range = "No match found"
print(f"{name} falls within masses: {mass_range}")
# print(f"Matching masses: {masses_in_box}")  # Debugging line

# Final output
# print("\nCepheid Data:")
for name, data in cepheids.items():
    print(f"{name}: Teff = {data['Teff']:.3f} ± {data['del_T']:.3f}, "
          f"Lum = {data['Lum']:.3f} ± {data['del_L']:.3f}")


In [ ]:
# Create HRD plot
fig, ax = plt.subplots(figsize=(14, 8))
colors = ['orange', 'yellow', 'green', 'teal']
for i, (mass, data) in enumerate(stellar_tracks.items()):
    ax.plot(data["Teff"], data["log_L"], color=colors[i % len(colors)], label=f'{mass}M')

    # Identify the ZAMS (First point in track)
    ZAMS_index = 0  # First point in the track
    Teff_ZAMS = data["Teff"][ZAMS_index]
    log_L_ZAMS = data["log_L"][ZAMS_index]

    # Annotate mass next to the ZAMS point
    ax.annotate(f"{mass:.1f}M", (Teff_ZAMS, log_L_ZAMS), textcoords="offset points",
                xytext=(-30, -10), ha='left', color=colors[i % len(colors)], fontsize=10)

# Plot instability strip
blue_left_edge = [3.91, 3.76]
blue_right_edge = [2.4, 4.5]
red_left_edge = [3.79, 3.65]
red_right_edge = [2.4, 4.5]
ax.plot(blue_left_edge, blue_right_edge, 'b', label="Blue Edge")
ax.plot(red_left_edge, red_right_edge, 'r', label="Red Edge")

for name, data in cepheids.items():
    ax.scatter(data["Teff"], data["Lum"], color=data["Color"], marker='X', label=f"{name}")
    ax.errorbar(data["Teff"], data["Lum"], xerr=data["del_T"], yerr=data["del_L"], color=data["Color"], elinewidth=20)

# Formatting
ax.invert_xaxis()
ax.set_xlabel('Log T_eff (K)', color='white')
ax.set_ylabel('Log L (Solar Luminosity)', color='white')
ax.set_facecolor("black")
fig.patch.set_facecolor('black')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
# ax.legend(loc='upper left', fontsize=10)
plt.title('Cepheid Evolution HR Diagram with Estimated Masses', color='white')
plt.show()
